In [1]:

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the dataset
file_path = 'https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv'
df = pd.read_csv(file_path)

# Drop unnecessary columns
df = df.drop(['EIN', 'NAME'], axis=1)

# Replace rare categories in APPLICATION_TYPE
application_type_counts = df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < 500].index.tolist()
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(application_types_to_replace, "Other")

# Replace rare categories in CLASSIFICATION
classification_counts = df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 100].index.tolist()
df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(classifications_to_replace, "Other")

# Convert categorical data to numeric
df_encoded = pd.get_dummies(df, columns=df.select_dtypes(include=['object']).columns)

# Split the data into features and target
target = 'IS_SUCCESSFUL'
X = df_encoded.drop(columns=[target]).values
y = df_encoded[target].values

# Transform y to a vertical vector
y = y.reshape(-1,1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [2]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_optimized = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_optimized.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=49))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_optimized.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_optimized.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_optimized.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_optimized

In [6]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=3)

Reloading Tuner from .\untitled_project\tuner0.json


In [7]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [8]:
# Get best model hyperparameters 1
best_hyper1 = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 9,
 'num_layers': 2,
 'units_0': 3,
 'units_1': 7,
 'units_2': 5,
 'units_3': 9,
 'units_4': 9,
 'units_5': 1,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 7,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0039'}

In [20]:
# Define the model using the given hyperparameters for optimization attempt #1
model1 = tf.keras.Sequential()

# Input layer and first hidden layer
model1.add(tf.keras.layers.Dense(units=9, activation='tanh', input_dim=X_train_scaled.shape[1]))

# Add subsequent layers based on num_layers
num_layers = 2
units = [3, 7, 5, 9, 9, 1]  # Extracted from 'units_X' in the hyperparameters

for i in range(num_layers):
    model1.add(tf.keras.layers.Dense(units=units[i], activation='tanh'))

# Output layer
model1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model1 = model1.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 505us/step - accuracy: 0.6666 - loss: 0.6227
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step - accuracy: 0.7245 - loss: 0.5670
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 527us/step - accuracy: 0.7258 - loss: 0.5614
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.7262 - loss: 0.5540
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step - accuracy: 0.7312 - loss: 0.5515
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.7353 - loss: 0.5471
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 504us/step - accuracy: 0.7311 - loss: 0.5504
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step - accuracy: 0.7287 - loss: 0.5505
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 498us/step - accuracy: 0.7263 - loss: 0.5518
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 491us/step - accuracy: 0.7344 - loss: 0.5444
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step - accuracy: 0.7311 - loss: 0.5485
Epoch 12/100
804/80

In [21]:
# Evaluate the model using the test data
model1_loss, model1_accuracy = model1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model1_loss}, Accuracy: {model1_accuracy}")

268/268 - 0s - 725us/step - accuracy: 0.7329 - loss: 0.5512
Loss: 0.5511971712112427, Accuracy: 0.7329446077346802


In [25]:
# Try optimization by changing starter_code model activation function to tanh for optimization 2
input_features = X_train.shape[1]  
model2 = tf.keras.models.Sequential()

# First hidden layer
model2.add(tf.keras.layers.Dense(units = 100, activation = 'tanh', input_dim = input_features))

# Second hidden layer
model2.add(tf.keras.layers.Dense(units = 50, activation = 'tanh'))

# Output layer
model2.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
       
# Check the structure of the model
model2.summary()

# Compile the model
model2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model2 = model2.fit(X_train_scaled, y_train, epochs = 100)

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_29 (Dense)                     │ (None, 100)                 │           5,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 50)                  │           5,050 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,101 (39.46 KB)

 Trainable params: 10,101 (39.46 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 520us/step - accuracy: 0.7125 - loss: 0.5846
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.7284 - loss: 0.5548
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 490us/step - accuracy: 0.7311 - loss: 0.5509
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step - accuracy: 0.7271 - loss: 0.5500
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7295 - loss: 0.5500
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7306 - loss: 0.5488
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 515us/step - accuracy: 0.7306 - loss: 0.5472
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 493us/step - accuracy: 0.7342 - loss: 0.5432
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.7288 - loss: 0.5466
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 497us/step - accuracy: 0.7346 - loss: 0.5427
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step - accuracy: 0.7342 - loss: 0.5431
Epoch 12/100
804/80

In [26]:
# Evaluate the model using the test data
model2_loss, model2_accuracy = model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model2_loss}, Accuracy: {model2_accuracy}")

268/268 - 0s - 724us/step - accuracy: 0.7291 - loss: 0.5596
Loss: 0.5595636367797852, Accuracy: 0.7290962338447571


In [28]:
# Define the model using the given hyperparameters with more epochs for optimization attempt 3
model3 = tf.keras.Sequential()

# Input layer and first hidden layer
model3.add(tf.keras.layers.Dense(units=9, activation='tanh', input_dim=X_train_scaled.shape[1]))

# Add subsequent layers based on num_layers
num_layers = 2
units = [3, 7, 5, 9, 9, 1]  # Extracted from 'units_X' in the hyperparameters

for i in range(num_layers):
    model3.add(tf.keras.layers.Dense(units=units[i], activation='tanh'))

# Output layer
model3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
model3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
fit_model3 = model3.fit(X_train_scaled, y_train, epochs = 150)

Epoch 1/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 525us/step - accuracy: 0.6627 - loss: 0.6294
Epoch 2/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 548us/step - accuracy: 0.7227 - loss: 0.5686
Epoch 3/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/step - accuracy: 0.7244 - loss: 0.5614
Epoch 4/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.7276 - loss: 0.5564
Epoch 5/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step - accuracy: 0.7315 - loss: 0.5510
Epoch 6/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.7309 - loss: 0.5505
Epoch 7/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - accuracy: 0.7298 - loss: 0.5529
Epoch 8/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step - accuracy: 0.7359 - loss: 0.5470
Epoch 9/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.7255 - loss: 0.5528
Epoch 10/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 528us/step - accuracy: 0.7339 - loss: 0.5452
Epoch 11/150
804/804 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.7313 - loss: 0.5502
Epoch 12/150
804/80

In [29]:
# Evaluate the model using the test data
model3_loss, model3_accuracy = model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model3_loss}, Accuracy: {model3_accuracy}")

268/268 - 0s - 1ms/step - accuracy: 0.7280 - loss: 0.5511
Loss: 0.5511334538459778, Accuracy: 0.7280466556549072


In [ ]:
# Save the optimized model to an HDF5 file
model1_optimized.save("AlphabetSoupCharity_Optimization.h5")

# Confirm the model has been saved
print("Optimized model saved to 'AlphabetSoupCharity_Optimization.h5'")
